In [1]:
import pandas as pd
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

2025-04-20 20:46:28.467273: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-20 20:46:28.503122: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-20 20:46:28.745415: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-20 20:46:32.678719: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../data/data.csv')
df.head()

,Date,BNB Open,BTC Close,BTC Open,DJI Open,DOGE Open,DXY Open,ETH Open,Gold Open,NASDAQ Open,...,US30 Open,BTC Active Addr Cnt,BTC Addr Cnt with ≥ .001% Supply,BTC Exchange Withdrawals,BTC Exchange Deposits,BTC Miner Supply,BTC Miner Revenue,BTC Difficulty,BTC Mean Block Size,BTC Total Fees
0,2014-09-18,1.99,424.3,461.1,17163.73,0.002803,84.80,11.2,1223.1,4084.51,...,3.371,191063,8893,9.743524e+06,1.065752e+07,9.222782e+08,2.127475e+06,2.982973e+10,236099.989247,5494.516757
1,2014-09-19,1.99,397.7,424.3,17267.21,0.002803,84.28,11.2,1225.6,4113.02,...,3.358,190890,8897,1.359548e+07,1.696480e+07,8.630416e+08,1.915503e+06,2.982973e+10,233777.391061,5214.342615
2,2014-09-20,1.99,411.5,397.7,17267.21,0.002803,84.28,11.2,1225.6,4113.02,...,3.358,172255,8890,1.694158e+07,1.770344e+07,7.968963e+08,1.688158e+06,2.982973e+10,224258.415205,4261.732292
3,2014-09-21,1.99,401.6,411.5,17271.71,0.002803,84.78,11.2,1216.4,4091.87,...,3.279,169941,8919,8.728492e+06,9.387692e+06,8.324557e+08,1.812817e+06,2.982973e+10,210502.045455,4235.264753
4,2014-09-22,1.99,404.1,401.6,17271.71,0.002803,84.78,11.2,1216.4,4091.87,...,3.279,200712,8905,1.089809e+07,1.050540e+07,8.117053e+08,1.645297e+06,2.982973e+10,214022.042683,3638.427361


In [3]:
# Load the Bi-LSTM model
model = load_model('../artifacts/bi_lstm_model.h5')

In [4]:
test_size =int(len(df) * 0.15);

In [5]:
df_test = df.tail(test_size)
df_test

,Date,BNB Open,BTC Close,BTC Open,DJI Open,DOGE Open,DXY Open,ETH Open,Gold Open,NASDAQ Open,...,US30 Open,BTC Active Addr Cnt,BTC Addr Cnt with ≥ .001% Supply,BTC Exchange Withdrawals,BTC Exchange Deposits,BTC Miner Supply,BTC Miner Revenue,BTC Difficulty,BTC Mean Block Size,BTC Total Fees
3288,2023-09-19,216.0,27209.2,26759.0,34571.84,0.061987,105.08,1636.88,1936.7,15152.53,...,4.385,1125172,8909,7.431922e+08,6.590047e+08,4.844965e+10,2.735718e+07,5.415014e+13,1.726593e+06,1.261666e+06
3289,2023-09-20,217.2,27125.0,27209.2,34575.50,0.062687,105.11,1643.14,1934.7,15225.34,...,4.428,1035954,8916,7.671415e+08,6.632496e+08,4.927249e+10,2.493483e+07,5.711987e+13,1.675497e+06,1.291754e+06
3290,2023-09-21,214.3,26570.0,27125.2,34332.23,0.062624,105.44,1622.28,1932.8,14813.49,...,4.446,1062056,8915,8.002886e+08,6.648565e+08,4.911485e+10,2.603134e+07,5.711987e+13,1.668772e+06,1.272111e+06
3291,2023-09-22,210.8,26581.9,26569.5,34077.08,0.061311,105.38,1584.03,1921.8,14756.62,...,4.576,1078476,8921,5.869487e+08,5.403715e+08,4.809891e+10,2.417754e+07,5.711987e+13,1.639065e+06,1.259540e+06
3292,2023-09-23,211.0,26579.2,26581.7,34077.08,0.061525,105.38,1593.10,1921.8,14756.62,...,4.576,978848,8921,5.343442e+08,6.316914e+08,4.811751e+10,2.266748e+07,5.711987e+13,1.619526e+06,1.236432e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3863,2025-04-16,579.5,84032.2,83648.1,40179.49,0.153564,100.06,1587.39,3248.4,18475.76,...,4.788,764894,10993,2.412105e+09,2.068111e+09,1.488710e+11,3.813252e+07,1.215078e+14,1.511157e+06,4.804138e+05
3864,2025-04-17,582.7,84940.0,84032.2,39745.58,0.154725,99.23,1577.85,3357.5,18369.90,...,4.751,715957,10993,2.459224e+09,1.981666e+09,1.497325e+11,3.941116e+07,1.215078e+14,1.623463e+06,4.910781e+05
3865,2025-04-18,590.4,84474.6,84947.7,39745.58,0.155837,99.39,1584.61,3341.3,18369.90,...,4.751,734302,10995,2.819279e+09,2.568826e+09,1.511429e+11,4.112598e+07,1.215078e+14,1.689454e+06,5.137650e+05
3866,2025-04-19,591.6,85068.1,84472.4,39745.58,0.157574,99.39,1587.98,3341.3,18369.90,...,4.751,669757,11002,1.616599e+09,1.388562e+09,1.502457e+11,3.633370e+07,1.215078e+14,1.658516e+06,4.489202e+05


In [6]:
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

X = df_test.drop(['Date', 'BTC Close'], axis=1)
y = df_test['BTC Close']

scaler = MinMaxScaler(feature_range=(0, 1))
test_X = scaler.fit_transform(X)
test_y = scaler.fit_transform(y.values.reshape(-1, 1))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [7]:
prediction = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

inv_test_prediction = np.concatenate((prediction, test_X[:, 1:]), axis=1)
inv_test_prediction = scaler.inverse_transform(inv_test_prediction)
inv_test_prediction = inv_test_prediction[:,0]
df_test['Predicted Price'] = inv_test_prediction

19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step


/tmp/ipykernel_240875/349534206.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Predicted Price'] = inv_test_prediction


In [8]:
df_calculate = pd.DataFrame()
df_calculate['Date'] = pd.to_datetime(df_test['Date'])
df_calculate['Open'] = df_test['BTC Open']
df_calculate['Actual Close'] = df_test['BTC Close'].tail(test_size).values
df_calculate['Predicted Close'] = df_test['Predicted Price']

d = df_calculate.to_dict('records')

inital_shares = 1000.0/ d[0]['Open'] * 0.995
peak_value = 1000
trough_value  = 1000

trade_cash = 1000.0
trade_shares = 0
total_trades = 0
win_trades = 0
trade_peak_value = 1000
trade_trough_value = 1000
correct_buy_prediction = 0
correct_sell_prediction = 0
wrong_buy_prediction = 0
wrong_sell_prediction = 0

daily_reports = []
monthly_reports = []
profit_taken = 0
profit_taken_in_btc = 0

# Loop through each day, skip first day
for i in range(0, len(d)):
    data = d[i]

    if data['Predicted Close'] > data['Open']:
        if trade_cash > 0:
            trade_shares = (trade_cash / data['Open']) * 0.995
            trade_cash = 0
            total_trades += 1
            if data['Actual Close'] > data['Predicted Close']:
                win_trades += 1
        if data['Actual Close'] > data['Predicted Close']:
            correct_buy_prediction += 1
        else:
            wrong_buy_prediction += 1
      
    elif data['Predicted Close'] < data['Open']:
        if trade_shares > 0:
            trade_cash = trade_shares * data['Open'] * 0.995 # 0.3% trading fee and 0.2% slippage
            trade_shares = 0
            total_trades += 1
            if data['Actual Close'] < data['Predicted Close']:
                win_trades += 1
        if data['Actual Close'] < data['Predicted Close']:
            correct_sell_prediction += 1
        else:
            wrong_sell_prediction += 1

    # calculate peak and trough values
    buy_and_hold_value = inital_shares * data['Actual Close'] * 0.995
    peak_value = max(peak_value, buy_and_hold_value)
    trough_value = min(trough_value, buy_and_hold_value)

    trade_value = (trade_shares * data['Actual Close'] * 0.995) + trade_cash
    trade_peak_value = max(trade_peak_value, trade_value)
    trade_trough_value = min(trade_trough_value, trade_value)

    # print balances
    buy_and_hold_profit = (inital_shares * data['Actual Close'] * 0.995) - 1000
    bi_lstm_trade_profit =  ((trade_shares * data['Actual Close'] * 0.995) + trade_cash) - 1000

    daily_reports.append({
        'Date': data['Date'],
        'Trade Profit': bi_lstm_trade_profit,
        "Trade Profit Taken": profit_taken,
        'Total Trade Profit': bi_lstm_trade_profit + profit_taken,
        'Total Trade Profit (btc)': bi_lstm_trade_profit + (profit_taken_in_btc * data['Actual Close'] * 0.995),
        'Buy and Hold Profit': buy_and_hold_profit,
    })

    if data['Date'].day == 1:
        monthly_reports.append({
            'Date': data['Date'],
            'Trade Profit': bi_lstm_trade_profit,
            "Trade Profit Taken": profit_taken,
            'Total Trade Profit': bi_lstm_trade_profit + profit_taken,
            'Total Trade Profit (btc)': bi_lstm_trade_profit + (profit_taken_in_btc * data['Actual Close'] * 0.995),
            'Buy and Hold Profit': buy_and_hold_profit,
        })
    
    # check for profit everyday
    if trade_shares > 0 and trade_shares * data['Actual Close'] * 0.995 > 1000:
        # sell away anything above 1000
        trade_cash = trade_shares * data['Actual Close'] * 0.995 - 1000
        shares_to_sell = (trade_cash / data['Actual Close']) * 0.995
        trade_shares = trade_shares - shares_to_sell
        profit_taken += trade_cash
        profit_taken_in_btc +=  shares_to_sell
        trade_cash = 0

print('Total Trades:', total_trades)
print('Win Trades:', win_trades)
print('Win Ratio:', win_trades / total_trades if total_trades >0 else 0)
print('Loss Ratio:', (1 - (win_trades / total_trades)) if total_trades >0 else 0)
print('Correct Buy Prediction:', correct_buy_prediction)
print('Correct Sell Prediction:', correct_sell_prediction)
print('Wrong Buy Prediction:', wrong_buy_prediction)
print('Wrong Sell Prediction:', wrong_sell_prediction)
print("================================================")

df_monthly_reports = pd.DataFrame(monthly_reports)


trade_returns = np.array([report['Total Trade Profit'] for report in daily_reports])
buy_and_hold_returns = np.array([report['Buy and Hold Profit'] for report in daily_reports])

trade_avg_return = np.mean(trade_returns)
buy_and_hold_avg_return = np.mean(buy_and_hold_returns)

# Calculate the standard deviation of the returns for each strategy
trade_std_dev = np.std(trade_returns)
buy_and_hold_std_dev = np.std(buy_and_hold_returns)

# Set the risk-free rate (4.8% MPY) (21 trading days) (0.223% DPY)
risk_free_returns = 0.223 / 100

# Calculate the Sharpe ratio for each strategy
trade_sharpe_ratio = (trade_avg_return - risk_free_returns) / trade_std_dev
buy_and_hold_sharpe_ratio = (buy_and_hold_avg_return - risk_free_returns) / buy_and_hold_std_dev

print(f"Buy and Hold Sharpe Ratio: {buy_and_hold_sharpe_ratio:.2f}")
print(f"Trade Sharpe Ratio: {trade_sharpe_ratio:.2f}")
print("================================================")

# print peak, trough and maximum drawdown value for each strategy
print('Buy and Hold Peak Value:', peak_value)
print('Buy and Hold Trough Value:', trough_value)
print('Buy and Hold Maximum Drawdown:', (trough_value - peak_value) / peak_value)
print("================================================")

print('Trade Peak Value:', trade_peak_value)
print('Trade Trough Value:', trade_trough_value)
print('Trade Maximum Drawdown:', (trade_trough_value - trade_peak_value) / trade_peak_value)
print("================================================")

# print last row of daily reports
df_daily_reports = pd.DataFrame(daily_reports)
print('Buy and Hold Profit:', df_daily_reports['Buy and Hold Profit'].iloc[-1])
print('Total Trade Profit:', df_daily_reports['Total Trade Profit'].iloc[-1])
print('Total Trade Profit (Profit taken in BTC):', df_daily_reports['Total Trade Profit (btc)'].iloc[-1])
print('Trade Profit Taken:', df_daily_reports['Trade Profit Taken'].iloc[-1])
print('Trade Profit Taken in BTC:', profit_taken_in_btc)

# save daily reports to csv
df_daily_reports.to_csv('./bi_lstm_daily_reports.csv', index=False)
df_monthly_reports.to_csv('./bi_lstm_monthly_reports.csv', index=False)

Total Trades: 46
Win Trades: 17
Win Ratio: 0.3695652173913043
Loss Ratio: 0.6304347826086957
Correct Buy Prediction: 42
Correct Sell Prediction: 53
Wrong Buy Prediction: 113
Wrong Sell Prediction: 372
Buy and Hold Sharpe Ratio: 1.81
Trade Sharpe Ratio: 2.63
Buy and Hold Peak Value: 3927.586304794648
Buy and Hold Trough Value: 970.0424706640756
Buy and Hold Maximum Drawdown: -0.7530181655130316
Trade Peak Value: 1091.3502381477795
Trade Trough Value: 842.681817084153
Trade Maximum Drawdown: -0.22785391194458546
Buy and Hold Profit: 2153.569450091558
Total Trade Profit: 617.5332647379055
Total Trade Profit (Profit taken in BTC): 1264.0798250478797
Trade Profit Taken: 693.1641884681604
Trade Profit Taken in BTC: 0.015796535333367532
